# Part 1 V2
This script is meant to be an update to the original Part 1, simplifying the functions. Some changes to original part 1 code will be needed. 

In [2]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Download All of the Data

In [43]:
# Information
DATASOURCE_IDS = ['39', '42', '99', 'SQLDS99']
j_drive_output_folder_path = 'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/'
geography_levels = ['mgra', 'region']

In [44]:
# download all of the data 
all_data = {}
for ds_id in DATASOURCE_IDS:
    temp_dict = {}
    for file in os.listdir(j_drive_output_folder_path):
        if (ds_id in file):
            temp_dict[file] = pd.read_csv(j_drive_output_folder_path + file)
    all_data[ds_id] = temp_dict

In [45]:
all_data.keys()

dict_keys(['39', '42', '99', 'SQLDS99'])

In [46]:
all_data['SQLDS99'].keys()

dict_keys(['mgra_SQLDS99_ind_QA.csv', 'region_SQLDS99_ind_QA.csv'])

# Both Files

In [7]:
def create_both_file(DS1, DS2, level, send_to_j_drive):
    df1 = all_data[DS1][f"{level}_DS{DS1}_ind_QA.csv"]
    df2 = all_data[DS2][f"{level}_DS{DS2}_ind_QA.csv"]

    # First check to see if shapes are identical
    if df1.shape != df2.shape:
        print(f"Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}")
    elif len(set(df1.columns).difference(set(df2.columns))) != 0:
        print(f"There is a difference in columns. The columns that exist in DF1 but not in DF2 are {[x for x in df1.columns if x not in df2.columns]}. The columns that exist in DF2 but not in DF1 are {[x for x in df2.columns if x not in df1.columns]}")
    elif len(set(df1['yr_id']).difference(set(df2['yr_id']))) != 0:
        print(f"There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are {[x for x in list(set(df1['yr_id'])) if x not in list(set(df2['yr_id']))]}. The columns that exist in DF2 but not in DF1 are {[x for x in list(set(df2['yr_id'])) if x not in list(set(df1['yr_id']))]}")
    

    if level == 'region':
        output = df1.merge(df2, on=['year'], how='left', suffixes=[f'_DS{DS1}', f'_DS{DS2}'])
        output = output.groupby(['year']).sum()
    else:
        output = df1.merge(df2, on=[f'{level}','year'], how='left', suffixes=[f'_DS{DS1}', f'_DS{DS2}'])
        output = output.groupby([f'{level}', 'year']).sum()

    if send_to_j_drive == 'Y':
        output.to_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/both_files/' + f"{level}_both_DS{DS1}_DS{DS2}_QA.csv", index=True) 

    return output

In [31]:
both_file = create_both_file(DS1='35', DS2='41', level='mgra', send_to_j_drive='N')

Shapes are not identical. DF1 is shape (299026, 145) and DF2 is shape (299026, 148)


In [32]:
both_file

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra  year                                                                   
1     2016      3331       19          19           0           0       18   
      2018      3331       19          19           0           0       18   
      2020      3331       19          19           0           0       18   
      2023      3331       20          20           0           0       18   
      2025      3331       20          20           0           0       18   
...              ...      ...         ...         ...         ...      ...   
23002 2032      1254      120          20         100           0      100   
      2035      1254      120          20         100           0      100   
      2040      1254      120          20         100           0      106   
      2045      1254      120          20         100           0      107   
      2050      1254      120          20         100           0      109   

            hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra  year                                                   ...   
1     2016          18           0           0            0  ...   
      2018          18           0           0            0  ...   
      2020          18           0           0            0  ...   
      2023          18           0           0            0  ...   
      2025          18           0           0            0  ...   
...                ...         ...         ...          ...  ...   
23002 2032          14          86           0            0  ...   
      2035          14          86           0            0  ...   
      2040          16          90           0            0  ...   
      2045          17          90           0            0  ...   
      2050          17          92           0            0  ...   

            American Indian_DS41  Asian_DS41  Black_DS41  Hispanic_DS41  \
mgra  year                                                                
1     2016                   0.0         1.0         0.0            1.0   
      2018                   0.0         1.0         0.0           11.0   
      2020                   0.0         1.0         0.0            4.0   
      2023                   0.0         0.0         0.0            0.0   
      2025                   0.0         1.0         0.0            4.0   
...                          ...         ...         ...            ...   
23002 2032                   0.0         0.0         0.0            0.0   
      2035                   2.0        44.0         3.0           52.0   
      2040                   1.0        69.0         2.0           60.0   
      2045                   0.0        77.0         5.0           69.0   
      2050                   2.0        87.0         9.0           61.0   

            Other_DS41  Pacific Islander_DS41  Two or More_DS41  White_DS41  \
mgra  year                                                                    
1     2016         1.0                    0.0               2.0        36.0   
      2018         1.0                    0.0               2.0        35.0   
      2020         1.0                    0.0               6.0        31.0   
      2023         0.0                    0.0               0.0         0.0   
      2025         1.0                    0.0               2.0        34.0   
...                ...                    ...               ...         ...   
23002 2032         0.0                    0.0               0.0         0.0   
      2035         1.0                    1.0               3.0       133.0   
      2040         0.0                    1.0              14.0       109.0   
      2045         1.0                    0.0               9.0       102.0   
      2050         1.0                    1.0               6.0       101.0   

            Female_DS41  Male_DS41  
mgra  year                          
1     2016         23.0       18.0

In [15]:
all_data['39']['mgra_DS39_ind_QA.csv']

,mgra,taz,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,...,American Indian,Asian,Black,Hispanic,Other,Pacific Islander,Two or More,White,Female,Male
0,1,3331,19,19,0,0,18,18,0,0,...,0.0,1.0,0.0,1.0,1.0,0.0,2.0,36.0,23.0,18.0
1,2,3331,35,35,0,0,34,34,0,0,...,0.0,5.0,0.0,11.0,0.0,0.0,2.0,63.0,41.0,40.0
2,3,3358,52,52,0,0,52,52,0,0,...,0.0,5.0,0.0,11.0,0.0,0.0,0.0,95.0,57.0,54.0
3,4,3358,30,30,0,0,30,30,0,0,...,1.0,0.0,0.0,12.0,0.0,1.0,4.0,55.0,35.0,38.0
4,5,3358,28,28,0,0,28,28,0,0,...,1.0,2.0,1.0,8.0,0.0,0.0,1.0,50.0,29.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299021,22998,1290,90,90,0,0,87,87,0,0,...,2.0,61.0,2.0,29.0,0.0,0.0,9.0,81.0,94.0,90.0
299022,22999,1290,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299023,23000,1290,131,131,0,0,126,126,0,0,...,3.0,93.0,4.0,89.0,2.0,2.0,6.0,99.0,155.0,143.0
299024,23001,1254,83,83,0,0,81,81,0,0,...,0.0,59.0,4.0,39.0,0.0,1.0,3.0,88.0,98.0,96.0


In [18]:
all_data['99']['mgra_DS99_ind_QA.csv'].columns

Index(['mgra', 'taz', 'hs', 'hs_sf', 'hs_mf', 'hs_mh', 'hh', 'hh_sf', 'hh_mf',
       'hh_mh',
       ...
       'truckregiontype', 'district27', 'milestocoast', 'acres',
       'effective_acres', 'land_acres', 'MicroAccessTime', 'remoteAVParking',
       'refueling_stations', 'year'],
      dtype='object', length=108)

In [20]:
'year' in all_data['39']['mgra_DS39_ind_QA.csv'].columns

True

# Diff Files

In [ ]:
# From Eric's code it will be yr_id from CSV it will be 'year'

In [10]:
# For Eric's SQL Code 
def create_diff_file(DS1, DS2, level, send_to_j_drive):
    df1 = all_data[DS1][f"{level}_DS{DS1}_ind_QA.csv"]
    df2 = all_data[DS2][f"{level}_DS{DS2}_ind_QA.csv"]

    # First check to see if shapes are identical
    if df1.shape != df2.shape:
        print("Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}")

    # Check if different columns are present amongst datasets, if so, manipulate so that only the shared columns are present
    if len(set(df1.columns).difference(set(df2.columns))) != 0:
        print(f"There is a difference in columns. The columns that exist in DF1 but not in DF2 are {[x for x in df1.columns if x not in df2.columns]}. The columns that exist in DF2 but not in DF1 are {[x for x in df2.columns if x not in df1.columns]}")
        # Ensure each data frame has the same columns
        df1 = df1[[x for x in df1.columns if x in df2.columns]]
        df2 = df2[[x for x in df1.columns if x in df2.columns]]

    # Check if different years are present amongst datasets, if so, manipulate so that only the shared years are compared
    if len(set(df1['yr_id']).difference(set(df2['yr_id']))) != 0:
        print(f"There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are {[x for x in list(set(df1['yr_id'])) if x not in list(set(df2['yr_id']))]}. The columns that exist in DF2 but not in DF1 are {[x for x in list(set(df2['yr_id'])) if x not in list(set(df1['yr_id']))]}")
        shared_years = [x for x in list(set(df1['yr_id'])) if x in list(set(df2['yr_id']))]
        df1 = df1[df1['yr_id'].isin(shared_years)]
        df2 = df2[df2['yr_id'].isin(shared_years)]
    
    df1 = df1.groupby([f'{level}', 'yr_id']).sum()
    df2 = df2.groupby([f'{level}', 'yr_id']).sum()

    diff_df = df2 - df1


    if send_to_j_drive == 'Y':
        diff_df.to_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs/' + f"{level}_diff_DS{DS2}_minus_DS{DS1}_QA.csv", index=True) 

    return diff_df

In [39]:
all_data['99']['region_DS99_ind_QA.csv']

,year,taz,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,...,hotelroomtotal,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations
0,2016,53370775,1190554,717626,430716,42212,1134848,687509,407158,40181,...,56646,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
1,2018,53370775,1205852,722026,441719,42107,1147635,691545,416096,39994,...,61917,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
2,2020,53370775,1226462,726659,457696,42107,1166240,695800,430412,40028,...,61917,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
3,2023,53370775,1262957,734381,486469,42107,1197072,701792,455124,40156,...,63707,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
4,2025,53370775,1288217,739817,506293,42107,1219745,706172,473375,40198,...,64889,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
5,2026,53370775,1300847,742347,516393,42107,1231007,708588,482210,40209,...,65379,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
6,2029,53370775,1338737,748717,547913,42107,1264151,714583,509347,40221,...,66938,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
7,2030,53370775,1351367,750795,558465,42107,1274948,716321,518403,40224,...,67428,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
8,2032,53370775,1376162,754407,579648,42107,1296193,719148,536815,40230,...,68233,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
9,2035,53370775,1409867,760440,607320,42107,1327588,724400,562943,40245,...,69454,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699


In [50]:
# For my CSV Code 
# For my CSV Code 
def create_diff_file(DS1, DS2, level, send_to_j_drive):
    df1 = all_data[DS1][f"{level}_DS{DS1}_ind_QA.csv"]
    df2 = all_data[DS2][f"{level}_DS{DS2}_ind_QA.csv"]

    # First check to see if shapes are identical
    if df1.shape != df2.shape:
        print("Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}")

    # Check if different columns are present amongst datasets, if so, manipulate so that only the shared columns are present
    if len(set(df1.columns).difference(set(df2.columns))) != 0:
        print(f"There is a difference in columns. The columns that exist in DF1 but not in DF2 are {[x for x in df1.columns if x not in df2.columns]}. The columns that exist in DF2 but not in DF1 are {[x for x in df2.columns if x not in df1.columns]}")
        # Ensure each data frame has the same columns
        df1 = df1[[x for x in df1.columns if x in df2.columns]]
        df2 = df2[[x for x in df1.columns if x in df2.columns]]

    # Check if different years are present amongst datasets, if so, manipulate so that only the shared years are compared
    if len(set(df1['year']).difference(set(df2['year']))) != 0:
        print(f"There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are {[x for x in list(set(df1['year'])) if x not in list(set(df2['year']))]}. The columns that exist in DF2 but not in DF1 are {[x for x in list(set(df2['year'])) if x not in list(set(df1['year']))]}")
        shared_years = [x for x in list(set(df1['year'])) if x in list(set(df2['year']))]
        df1 = df1[df1['year'].isin(shared_years)]
        df2 = df2[df2['year'].isin(shared_years)]

    if level == 'region':
        diff_df = df2 - df1
    else:
        df1 = df1.groupby([f'{level}', 'year']).sum()
        df2 = df2.groupby([f'{level}', 'year']).sum()

        diff_df = df2 - df1       

    if send_to_j_drive == 'Y':
        diff_df.to_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/diff_files/' + f"{level}_diff_DS{DS2}_minus_DS{DS1}_QA.csv", index=True) 

    return diff_df

In [49]:
result = create_diff_file(DS1='SQLDS99', DS2='99', level='mgra', send_to_j_drive='Y')

Shapes are not identical. DF1 is shape {df1.shape} and DF2 is shape {df2.shape}
There is a difference in the years of these dataframes. The years that exist in DF1 but not in DF2 are [2017]. The columns that exist in DF2 but not in DF1 are []


In [26]:
result

taz  hs  hs_sf  hs_mf  hs_mh  hh  hh_sf  hh_mf  hh_mh  gq_civ  \
mgra  year                                                                  
1     2016    0   0      0      0      0   0      0      0      0       0   
      2018    0   0      0      0      0   0      0      0      0       0   
      2020    0   0      0      0      0   0      0      0      0       0   
      2023    0   0      0      0      0   0      0      0      0       0   
      2025    0   0      0      0      0   0      0      0      0       0   
...         ...  ..    ...    ...    ...  ..    ...    ...    ...     ...   
23002 2032    0   0      0      0      0   0      0      0      0       0   
      2035    0   0      0      0      0   1     -1      2      0       0   
      2040    0   0      0      0      0   1     -1      2      0       0   
      2045    0   0      0      0      0   1     -1      2      0       0   
      2050    0   0      0      0      0   1     -1      2      0       0   

            ...  luz_id  truckregiontype  district27  milestocoast  acres  \
mgra  year  ...                                                             
1     2016  ...       0                0           0           0.0    0.0   
      2018  ...       0                0           0           0.0    0.0   
      2020  ...       0                0           0           0.0    0.0   
      2023  ...       0                0           0           0.0    0.0   
      2025  ...       0                0           0           0.0    0.0   
...         ...     ...              ...         ...           ...    ...   
23002 2032  ...       0                0           0           0.0    0.0   
      2035  ...       0                0           0           0.0    0.0   
      2040  ...       0                0           0           0.0    0.0   
      2045  ...       0                0           0           0.0    0.0   
      2050  ...       0                0           0           0.0    0.0   

            effective_acres  land_acres  MicroAccessTime  remoteAVParking  \
mgra  year                                                                  
1     2016              0.0         0.0                0                0   
      2018              0.0         0.0                0                0   
      2020              0.0         0.0                0                0   
      2023              0.0         0.0                0                0   
      2025              0.0         0.0                0                0   
...                     ...         ...              ...              ...   
23002 2032              0.0         0.0                0                0   
      2035              0.0         0.0                0                0   
      2040              0.0         0.0                0                0   
      2045              0.0         0.0                0                0   
      2050              0.0         0.0                0                0   

            refueling_stations  
mgra  year                      
1     2016                   0  
      2018                   0  
      2020                   0  
      2023                   0  
      2025                   0  
...                        ...  
23002 2032                   0  
      2035                   0  
      2040                   0  
      2045                   0  
      2050                   0  

[299026 rows x 106 columns]